In [1]:
from datasets import Dataset

train_dataset = Dataset.from_file("../../llm-model/Llama2-law-finetuning-da/autotrain-data/train/data-00000-of-00001.arrow")
train_dataset

/home/crysis/anaconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] Failed to open local file '../../llm-model/Llama2-law-finetuning-da/autotrain-data/train/data-00000-of-00001.arrow'. Detail: [errno 2] No such file or directory

In [33]:
train_dataset['instruction']

['동거 중인데 갑자기 헤어지자는 통보를 받았어요. 사실혼인 경우에도 위자료를 받을 수 있나요?',
 '이혼이 취소될 수 있나요?',
 '이혼해도 자녀를 만날 수 있나요?',
 '이혼한 후에 자녀의 성과 본을 저의 성과 본으로 바꿀 수 있나요?',
 '중학생 딸아이가 학교 숙제로 유언장을 작성했는데, 이 유언장이 법적으로 효력 있는 것인지 궁금해 합니다.',
 '아버지가 빚진 부분에 대해서만 상속포기를 할 수 있나요?',
 '아버지는 저에게 모든 재산을 물려준다는 내용의 유언장을 작성했습니다.\n그러던 중 형이 아버지에게 제가 아버지의 재산을 몰래 팔아치웠다고 거짓말을 해서 아버지는 유산을 형에게 모두 물려준다는 내용의 유언장을 새로 작성했습니다.\n정말 형이 모든 재산을 물려받나요?',
 '동거 중인데 헤어지기로 합의했어요. 둘 사이에 아이가 있는데 헤어진 후 아이의 아빠에게 양육비를 받을 수 있나요?',
 '자녀를 데리고 재혼합니다. 이 자녀가 재혼하는 배우자의 자녀로 인정받을 수 있나요?',
 '자녀를 데리고 재혼했는데 전(前) 배우자가 사망했어요. 이 자녀가 전 배우자의 재산을 상속받을 수 있나요?',
 '아버지가 갑자기 돌아가셔서 부동산, 예금 등에 대한 정확한 재산상황을 알지 못합니다. 이런 것들은 어떻게 확인할 수 있나요?',
 '보험금 지급 청구권은 상속재산에 포함되나요?',
 '부모님께 상속받을 재산보다 빚이 더 많은데, 이 빚도 무조건 상속되나요?',
 '상속의 한정승인과 상속포기의 차이는 무엇인가요?',
 '이혼소송 진행 중에 부인이 교통사고로 사망했습니다.\n이혼판결이 확정되기 전에 사망했어도 남편이 상속인이 될 수 있나요?',
 '자녀를 데리고 재혼했는데 재혼한 배우자가 사망했어요. 제가 데려온 자녀가 재혼 배우자의 재산을 상속받을 수 있나요?',
 '혼인신고 없이 살고 있는데 부인이 사망했어요. 부인의 재산을 상속받을 수 있나요?',
 '혼인신고를 하기 전에 남편이 죽었어요. 지금 살고 있는 전셋집이 남편명의로 되어 있는데, 상속받아서

In [63]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import warnings
warnings.filterwarnings('ignore')


model_id = "TinyPixel/Llama-2-7B-bf16-sharded"
peft_model_id = "../../llm-model/Llama2-law-finetuning-da/checkpoint-11520"

config = PeftConfig.from_pretrained(peft_model_id)
bnb_confg = BitsAndBytesConfig(
    load_in_8bit=False,
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
    llm_int8_enable_fp32_cpu_offload=False,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_confg, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: %s ### Response: "

def gen(x):
    q = prompt % (x,)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=4096,
        early_stopping=True,
        do_sample=True,
        output_scores=True,
    )
    return tokenizer.decode(gened[0]).replace(q, "")

Loading checkpoint shards: 100%|██████████| 14/14 [00:04<00:00,  3.24it/s]


In [75]:
gen_predict = gen(f"{train_dataset['instruction'][1]}")
print(train_dataset['instruction'][1])

이혼이 취소될 수 있나요?


In [76]:
gen_predict

'<s> 이혼이 취소될 수 있습니다. 혼인관계가 면책되는 취소사유에는 타당자격을 확보한 후의 신고청이 없는 경우, 타당자격을 확보한 후 합의관 또는 재판부의 이혼법원에 합의청을 하지 않은 경우, 합의관 또는 재판부의 이혼법원에 합의청한 경우에도 이혼취소소송을 제기할 수 있습니다. 다만, 합의관 또는 재판부의 이혼법원에 합의청을 하여 이혼매각을 다투고 이혼하는 피자격 세력이 약탄적인 경우에는 이혼이 취소되어도 대법원상고소청이 배제될 뿐입니다. 따라서 이혼소송이 이혼취소소송과 동시에 제기되는 경우에는 이혼이 취소되는 것이 확정될 뿐입니다. ◇ 이혼소송을 제기할 수 있는 취소사유 ☞ 혼인관계가 종료되지 않은 경우로서 타당자격을 확보한 후에 신고청이 없는 경우 ☞ 혼인관계가 종료된 경우 UTV(부부 사이에 법률후견소속의 변호사가 중복소속하지 않는 의법상 혼인관계를 한 지 하의 외국인이 대한법원을 통해 대한민국 국적을 취득했음에도 불구하고 대한민국 국적법에 따라 부부한도를 지원하여 그 밖에 대한법원이 인정한 한도 내에서는 혼인 생활을 하는 사람 ☞ 타당자격을 확보한 후 합의관 또는 재판부의 이혼법원에 합의청을 하지 않은 경우 ☞ 타당자격을 확보한 후 합의관 또는 재판부의 이혼법원에 합의청한 경우 ◇ 이혼소송이 어떻게 전개되나요?</s>'

In [78]:
refer_sentence = train_dataset['output'][1]
refer_sentence

'재판상 이혼은 재판절차를 거쳐 이혼판결이 선고된 것이므로 취소될 수 없지만, 협의이혼은 부부간 자유로운 의사에 따른 합의에 기초하므로 사기 또는 강박(强迫)으로 인해 이혼의 의사표시를 했다면 이혼이 취소될 수 있습니다. 이혼을 취소하려면 관할 가정법원에 이혼취소소송을 제기해야 합니다. 이혼취소판결이 확정되면 그 이혼은 처음부터 없었던 것과 같아지므로 취소판결 전에 다른 일방이 재혼했다면 그 재혼은 중혼(重婚)이 됩니다. ◇ 중혼 ☞ 중혼은 법률상 혼인관계가 둘 이상 존재하는 위법한 상태로서 혼인의 취소사유가 됩니다. 중혼을 이유로 재혼(後婚)이 취소되면 전 배우자와만 법률상의 부부관계가 유지되고, 재혼 배우자와의 법률상 부부관계는 종료됩니다.'

### Cosine Similarity

In [79]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [80]:
sentences = (gen_predict, refer_sentence)

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
cos_similar = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
print("코사인 유사도 측정")
print(cos_similar[0][0])

코사인 유사도 측정
0.05117647746204163


In [81]:
tfidf_normalized = tfidf_matrix/np.sum(tfidf_matrix)

euc_d_norm = euclidean_distances(tfidf_normalized[0:1],tfidf_normalized[1:2])
print("유클리디언 유사도 측정")
print(euc_d_norm[0][0])

유클리디언 유사도 측정
0.08973450572877936


### Jaccard Similarity

In [82]:
input_string = refer_sentence
answer_string = gen_predict

intersection_cardinality = len(set.intersection(*[set(answer_string), set(input_string)]))
union_cardinality = len(set.union(*[set(answer_string), set(input_string)]))
similar = intersection_cardinality / float(union_cardinality)

print(similar)

0.42073170731707316


### SequenceMatcher

In [83]:
import difflib

input_string = refer_sentence
answer_string = gen_predict

answer_bytes = bytes(answer_string, 'utf-8')
input_bytes = bytes(input_string, 'utf-8')
answer_bytes_list = list(answer_bytes)
input_bytes_list = list(input_bytes)

sm = difflib.SequenceMatcher(None, answer_bytes_list, input_bytes_list)
similar = sm.ratio()

print(similar)

0.11873245086241477
